In [ ]:
import bluemira.geometry as geo
from bluemira.codes import _freecadapi as cadapi
from bluemira.display import plot_2d, show_cad
from bluemira.display.plotter import DEFAULT_PLOT_OPTIONS
from operator import itemgetter

DEFAULT_PLOT_OPTIONS["plane"] = "xy"

# A simple tutorial for the bluemira geometric module

## 1. Creation of a bluemira wire

In [ ]:
pntslist = [(1.0, 1.0, 0.0), (0.0, 1.0, 0.0), (0.0, 0.0, 0.0), (1.0, 0.0, 0.0)]
bmwire = geo.tools.make_polygon(pntslist)
print(f"{bmwire=}")
plot_2d(bmwire)

### Same result using cadapi

In [ ]:
wire = cadapi.make_polygon(pntslist)
print(f"Freecad wire: {wire}, length: {wire.Length}, isClosed: {wire.isClosed()}")
bmwire = geo.wire.BluemiraWire(wire, "bmwire")
print(f"{bmwire=}")
plot_2d(bmwire)

## 2. Creation of a closed bluemira wire

In [ ]:
bmwire = geo.tools.make_polygon(pntslist, closed=True)
print(bmwire)
plot_2d(bmwire)

## 3. Make some operations on bluemira wire

In [ ]:
ndiscr = 10

### 3.1 Discretize in {ndiscr} points

In [ ]:
points = bmwire.discretize(ndiscr)
print(f"{points}")
plot_2d(points)

### 3.2 Discretize considering the edges

In [ ]:
points = bmwire.discretize(ndiscr, byedges=True)
print(f"{points=}")
plot_2d(points)

## 4. Creation of a bluemira face

In [ ]:
bmface = geo.face.BluemiraFace(bmwire, "bmface")
print(f"{bmface=}")
plot_2d(bmface)

## 5. Scaling a bluemira wire

**NOTE**: scale function modifies the original object

### 5.1 Scale a BluemiraWire

In [ ]:
print(f"Original object: {bmwire}")
bmwire.scale(2)
print(f"Scaled object: {bmwire}")

**NOTE**: since bmface is connected to bmwire, a scale operation on bmwire also affect
bmface

### 5.2 Scale a BluemiraFace

In [ ]:
print(f"Original object: {bmface}")
bmface.scale(2)
print(f"Scaled object: {bmface}")

## 6. Saving as a STEP file

In [ ]:
shapes = [bmwire._shape, bmface._shape]
print(shapes)
cadapi.save_as_STEP(shapes, "geo_shapes")

## 7. Closing a BluemiraWire

### 7.1 When boundary is list(Part.Wire)

In [ ]:
pntslist = [(1.0, 1.0, 0.0), (0.0, 1.0, 0.0), (0.0, 0.0, 0.0), (1.0, 0.0, 0.0)]
wire = cadapi.make_polygon(pntslist, closed=False)
bmwire_nc = geo.wire.BluemiraWire(wire)
print(f"Before close: {bmwire_nc}")
print(f"Boundary: {bmwire_nc.boundary}")
bmwire_nc.close()
print(f"After close: {bmwire_nc}")
print(f"Boundary: {bmwire_nc.boundary}")

### 7.2 When boundary is list(BluemiraWire)

In [ ]:
bmwire_nc = geo.wire.BluemiraWire(geo.wire.BluemiraWire(wire))
print(bmwire_nc)
bmwire_nc.close()
print(bmwire_nc)
print(bmwire_nc.boundary)

## 8. Translate

In [ ]:
bmface.translate((5.0, 2.0, 0.0))
plot_2d(bmface)
geo.tools.save_as_STEP([bmwire, bmface], "geo_translate")

## 9. Bezier spline curve

In [ ]:
pntslist = [(1.0, 1.0, 0.0), (0.0, 1.0, 0.0), (0.0, 0.0, 0.0), (1.0, 0.0, 0.0)]
bmwire_nc = geo.tools.make_bspline(pntslist, closed=False)
print(bmwire_nc)
plot_2d(bmwire_nc)

### Same result using cadapi

In [ ]:
wire = cadapi.make_bspline(pntslist, closed=False)
bmwire_nc = geo.wire.BluemiraWire(wire)
print(bmwire_nc)
plot_2d(bmwire_nc)
geo.tools.save_as_STEP([bmwire_nc], "geo_bspline")

## 10. Revolve

In [ ]:
bmsolid = geo.tools.revolve_shape(bmface, direction=(0.0, 1.0, 0.0))
show_cad(bmsolid)
geo.tools.save_as_STEP([bmsolid], "geo_revolve")

## 11. Face and solid with hole

In [ ]:
pntslist_out = [(1.0, 1.0, 0.0), (0.0, 1.0, 0.0), (0.0, 0.0, 0.0), (1.0, 0.0, 0.0)]
delta = 0.25
pntslist_in = [
    (1.0 - delta, 1.0 - delta, 0.0),
    (0.0 + delta, 1.0 - delta, 0.0),
    (0.0 + delta, 0.0 + delta, 0.0),
    (1.0 - delta, 0.0 + delta, 0.0),
]
wire_out = geo.tools.make_polygon(pntslist_out, closed=True)
wire_in = geo.tools.make_polygon(pntslist_in, closed=True)
bmface = geo.face.BluemiraFace([wire_out, wire_in])
plot_2d(bmface)
geo.tools.save_as_STEP([bmface], "geo_face_with_hole")

In [ ]:
bmsolid = geo.tools.revolve_shape(bmface, direction=(0.0, 1.0, 0.0))
show_cad(bmsolid)
geo.tools.save_as_STEP([bmsolid], "geo_solid_with_hole")

## 12. Solid creation from Shell

In [ ]:
vertexes = [
    (0.0, 0.0, 0.0),
    (1.0, 0.0, 0.0),
    (1.0, 1.0, 0.0),
    (0.0, 1.0, 0.0),
    (0.0, 0.0, 1.0),
    (1.0, 0.0, 1.0),
    (1.0, 1.0, 1.0),
    (0.0, 1.0, 1.0),
]

# faces creation
faces = []
v_index = [
    (0, 1, 2, 3),
    (5, 4, 7, 6),
    (0, 4, 5, 1),
    (1, 5, 6, 2),
    (2, 6, 7, 3),
    (3, 7, 4, 0),
]
for ind, value in enumerate(v_index):
    wire = geo.tools.make_polygon(list(itemgetter(*value)(vertexes)), closed=True)
    faces.append(geo.face.BluemiraFace(wire, "face" + str(ind)))

# shell creation
shell = geo.shell.BluemiraShell(faces, "shell")

# solid creation from shell
solid = geo.solid.BluemiraSolid(shell, "solid")
print(solid)
show_cad(solid)
geo.tools.save_as_STEP([solid], "geo_cube")